# Live Demo

Live Demo script - Road following with traffic sign detection

Scripts are based and tested on image, "Yahboom_jetbot_64G_20200520"

Original source code from https://github.com/NVIDIA-AI-IOT/jetbot/

Modified for P-project, needs road following with traffic sign detection.


### 1. Load Trained Models
Load trained models for Road following and Traffic sign detection
>``best_steering_model_xy.pth`` from ``Train_Model_Road``

>``best_model_resnet18.pth`` from ``Train_Model_Image``

Model weights are located on the GPU device.

In [1]:
import torchvision
import torch
from servoserial import ServoSerial

device = torch.device('cuda')

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy.pth'))
model = model.to(device)
model = model.eval().half()

model_image = torchvision.models.resnet18(pretrained=False)
model_image.fc = torch.nn.Linear(512, 5)
model_image.load_state_dict(torch.load('best_model_resnet18.pth'))
model_image = model_image.to(device)
model_image = model_image.eval().half()

### 2. Creating the Pre-Processing Function
Define Pre-Processing function which can convert images from the camera format to

the neural network input format.

In [2]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

### 3. Display live camera feed
Initialize and display camera

>When SerialException occurs, open terminal and type ``sudo chmod 777 /dev/ttyTHS1``

In [3]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg
from servoserial import ServoSerial

camera = Camera.instance(width=224, height=224,capture_width=224, capture_height=224, fps=5)
servo_device = ServoSerial()
image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

def camservoInitFunction():
    global leftrightpulse, updownpulse
    leftrightpulse = 2048
    updownpulse = 2048
    servo_device.Servo_serial_control(1, 2150)
    time.sleep(0.1)
    servo_device.Servo_serial_control(2, 1300)

serial Open!


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### 4. Create robot instance
Create robot instance to drive the motors.

In [4]:
from jetbot import Robot

robot = Robot()

### 5. Create sliders to control jetbot
Create sliders to control jetbot.

>speed_gain_slider : Speed Control

>steering_gain_slider : Steering Gain Control

>steering_dgain_slider : Steering kd Control

>steering_bias_slider : Steering Bias control

In [5]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

### 6. Create sliders to monitor jetbot
Create sliders to monitor jetbot.

>x, y slider : Predicted x, y values

>steering, speed slider : Estimated steering, speed value

>free, lego, red, yellow, green slider : Estimated traffic signs

In [6]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, description='speed')

free_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation = 'vertical', description='free')
lego_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation = 'vertical',description='lego')
red_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation = 'vertical',description='red')
yellow_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation = 'vertical',description='yellow')
green_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation = 'vertical',description='green')

display(ipywidgets.HBox([x_slider, y_slider]))
display(ipywidgets.HBox([speed_slider, steering_slider]))
display(ipywidgets.HBox([free_slider, lego_slider, red_slider, yellow_slider, green_slider]))

### 7. Create neural network execution function
Create function that will get called whenever the camera's value changes.

1. Pre-process the camera image

2. Execute the neural network

3. Compute the approximate steering value

4. Control the motors using proportional / derivative control (PD)

5. By the index of max value in image_detect_list, movement logic changes.

In [7]:
import time

angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    traffic_signs = model_image(preprocess(image)).detach().float().cpu()
    traffic_signs = F.softmax(traffic_signs, dim=1)
    
    image_lego = float(traffic_signs.flatten()[4])
    image_red = float(traffic_signs.flatten()[1])
    image_yellow = float(traffic_signs.flatten()[2])
    image_green = float(traffic_signs.flatten()[0])
    image_free = float(traffic_signs.flatten()[3])
    
    image_detect_list = [0, 0, 0, 0, 0]
    image_detect_list[4] = image_lego
    image_detect_list[1] = image_red
    image_detect_list[2] = image_yellow
    image_detect_list[0] = image_green
    image_detect_list[3] = image_free
    image_detect = image_detect_list.index(max(image_detect_list))
    
    lego_slider.value = image_lego
    red_slider.value = image_red
    yellow_slider.value = image_yellow
    green_slider.value = image_green
    free_slider.value = image_free

    if (image_detect == 3): #free
        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    else:
        if (image_detect == 0): #green
            robot.forward(0.0)
        if (image_detect == 1): #red
            robot.forward(0.0)
        if (image_detect == 2): #yellow
            robot.forward(0.0)
        if (image_detect == 4): #lego
            robot.right(0.0)
    
    time.sleep(0.001)

execute({'new': camera.value})

### 8. Attach neural network function to the camera
Attach neural network function to the camera for processing

>This code will move the robot

In [8]:
camservoInitFunction()
camera.observe(execute, names='value')

184
b'\xff\xff\x01\x07\x03*\x08\x00\x00\n\xb8'
166
b'\xff\xff\x02\x07\x03*\x05\x14\x00\n\xa6'


### 9. Unlink camera
Reduce CPU usage

In [ ]:
camera_link.unlink()

### 10. Stop robot and camera

In [9]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)

robot.stop()

camera.stop()